# 2. Basic flow with a stationary body
In this notebook we will simulate the flow past a stationary body.

In [ ]:
using ViscousFlow

In [ ]:
using Plots

### The basic steps
From the previous notebook, we add one additional step:
* **Specify the problem**: Set the Reynolds number and free stream
* **Discretize**: Set up a solution domain, grid cell size, time step size
* ***Set up bodies***: *Create the body or bodies and specify their motions, if any*
* **Construct the system structure**: Create the operators that will be used to perform the simulation
* **Initialize**: Set the initial flow field and initialize the integrator
* **Solve**: Solve the flow field
* **Examine**: Examine the results

We will go through all of these here. For the examples we will carry out in this notebook, the first three steps need only be carried out once.

### Problem specification
Set the Reynolds number and free stream

In [ ]:
Re = 200 # Reynolds number
U = 1.0 # Free stream velocity
U∞ = (U,0.0);

### Discretize

In [ ]:
xlim = (-1.0,3.0)
ylim = (-1.5,1.5)

Δx, Δt = setstepsizes(Re,gridRe=4.0)

### Set up bodies
Here, we will set up a rectangle at 45 degrees angle of attack

In [ ]:
body = Rectangle(0.5,0.25,1.5Δx)

Transform the body to a specified initial position and orientation.

In [ ]:
cent = (0.0,0.0) # center of body
α = 45π/180 # angle
T = RigidTransform(cent,α)
T(body) # transform the body to the current configuration

In [ ]:
plot(body,xlim=xlim,ylim=ylim)

### Construct the system structure
This step is like the previous notebook, but now we also provide the body and the freestream:

In [ ]:
sys = NavierStokes(Re,Δx,xlim,ylim,Δt,body,freestream = U∞)

### Initialize
Now, we initialize with zero vorticity

In [ ]:
u0 = newstate(sys)

and create the integrator

In [ ]:
tspan = (0.0,1.0)
integrator = init(u0,tspan,sys)

### Solve
Now we are ready to solve the problem. Let's advance the solution to $t = 10$. This should take a minute or two, depending on the machine.

In [ ]:
@time step!(integrator,10.0)

### Examine
Let's look at the flow field at the end of this interval

In [ ]:
plot(
plot(vorticity(integrator),sys,title="Vorticity",clim=(-10,10),levels=range(-10,10,length=30), color = :RdBu,ylim=ylim),
plot(streamfunction(integrator),sys,title="Streamlines",ylim=ylim,color = :Black),
    size=(700,300)
    )

Now let's make a movie

In [ ]:
sol = integrator.sol;

In [ ]:
@gif for (u,t) in zip(sol.u,sol.t)
    plot(vorticity(u,sys,t),sys,clim=(-10,10),levels=range(-10,10,length=30), color = :RdBu)
end every 5

#### Compute the force history
To do this, we supply the solution history `sol`, the system `sys`, and the index of the body (1).

In [ ]:
fx, fy = force(sol,sys,1);

In [ ]:
plot(
plot(sol.t,2*fx,xlim=(0,Inf),ylim=(0,6),xlabel="Convective time",ylabel="\$C_D\$",legend=:false),
plot(sol.t,2*fy,xlim=(0,Inf),ylim=(-6,6),xlabel="Convective time",ylabel="\$C_L\$",legend=:false),
    size=(800,350)
)

The mean drag and lift coefficients are

In [ ]:
meanCD = 2*sum(fx)/length(fx)

In [ ]:
meanCL = 2*sum(fy)/length(fy)